# Refinitiv Data Platform Library for Python
## Delivery - EndPoint - Overview

This notebook provides an overview of using the Delivery Layer's Endpoint interface to retrieve Refinitiv Data Platform content directly from the Endpoint.

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
from refinitiv.data.delivery import omm_item_stream
import datetime
import json
import pandas as pd
%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop")   

You can also set a default in the **credentials** notebook

**Note**: You cannot currently perform Endpoint requests using a Deployed session

In [2]:
open_session()

## Defining an Endpoint
Defining an Endpoint
At the heart of the Endpoint Request interfaces, is the data endpoint URL.  For example, the following data services are represented by their endpoint URLs:

* Historical Pricing: https://api.refinitiv.com/data/historical-pricing/v1/views/events
* News Headlines: https://api.refinitiv.com/data/news/v1/headlines 
* Symbology Lookup: https://api.refinitiv.com/discovery/symbology/v1/lookup 
* Basic ESG scores: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic

When creating a request to retrieve data from an endpoint, the first step is to specify the endpoint URL, for example:

In [ ]:
esg_url = 'https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic'

### Defining Endpoint Request Properties
Depending on the service and how you choose to prepare your endpoint URL, the interfaces support the ability to specify additional properties to be included within your request.  The following request features are available:

* Query Parameters Endpoint: URLs supporting query strings are represented as field=value pairs.
Eg: https://api.refinitiv.com/data/environmental-social-governance/v2/views/basic?universe=IBM.N

* Path Parameters : In some cases, URLs define parameters that are embedded within the URL.  When defining the URL as part of your application, you must enclose the path parameter(s) within curly braces.
Eg: https://api.refinitiv.com/data/historical-pricing/v1/views/events/{universe}

* Header Parameters : Endpoints that enforce different content types or formats, typically require the specification within the header portion of the HTTP request.  For example, the endpoint to retrieve a news story offers the ability to specify the response format as either "text/html" or "application/json", which is specified within the HTTP header of the request.

* Body Parameters : Many of the RDP endpoints support the HTTP POST defining the ability to specify a body of data in the request.  The libraries support the ability to specify JSON details within the support endpoint request.  

The following simple code segments demonstrate the above mechanisms.

**Query Parameter:**

In [9]:
# Example request with Query Parameter - ESG
request_definition = rd.delivery.endpoint_request.Definition(
    url = esg_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    request_params = {"universe": "IBM.N"}
)
response = request_definition.get_data()
response.data.raw

NameError: name 'esg_url' is not defined

**Path Parameter:**

In [3]:
# Example request with Path parameter - News Story
# Specify our Endpoint URL
story_url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}'
# Specify our path parameter value
storyId = "urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2"
request_definition = rd.delivery.endpoint_request.Definition(
    url = story_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    path_parameters = {"storyId": storyId}
)
response = request_definition.get_data()
response.data.raw

{'newsItem': {'_conformance': 'power',
  '_guid': 'tag:reuters.com,2021-03-30:newsml_HKS9WjjsZ',
  '_standard': 'NewsML-G2',
  '_standardversion': '2.18',
  '_version': 2,
  'catalogRef': [{'_href': 'http://xml.media.reuters.com/g2-standards/catalogs/ReutersMedia_G2-Standards-Catalog_v1.xml'}],
  'rightsInfo': [{'copyrightHolder': {'_literal': ''},
    'copyrightNotice': [{}]}],
  'itemMeta': {'itemClass': {'_qcode': 'ninat:text', '_rtr:msgType': 'R'},
   'provider': {'_literal': 'reuters.com'},
   'versionCreated': {'$': '2021-03-30T14:09:48.681Z'},
   'firstCreated': {'$': '2021-03-30T14:09:09.503Z'},
   'pubStatus': {'_qcode': 'stat:usable'},
   'role': {'_qcode': 'itemRole:N'},
   'title': [{'$': 'Shenwan Hongyuan Group Co Ltd - An announcement has just been published by the issuer in the Chinese section of this website, a corresponding version of which may or may not be published in this section(with URL)'}],
   'signal': [{'_qcode': 'edStat:U'}],
   'expires': [{'$': '2022-04-30T

**Header Parameter:**

In [7]:
# Example request with Path + Header parameter - News Story
# Specify our Endpoint URL
story_url = 'https://api.refinitiv.com/data/news/v1/stories/{storyId}'
# Specify our path parameter value
storyId = "urn:newsml:reuters.com:20210330:nHKS9WjjsZ:2"
request_definition = rd.delivery.endpoint_request.Definition(
    url = story_url,
    method = rd.delivery.endpoint_request.RequestMethod.GET,
    path_parameters = {"storyId": storyId},
    header_parameters = {"accept": "text/html"}
)
response = request_definition.get_data()
response.data.raw

TypeError: __init__() got an unexpected keyword argument 'header_parameters'

**Body Parameter:**

In [8]:
# Example request with Body Parameter - Symbology Lookup
lookup_url = 'https://api.refinitiv.com/discovery/symbology/v1/lookup'
request_definition = rd.delivery.endpoint_request.Definition(
    url = lookup_url,
    method = rd.delivery.endpoint_request.RequestMethod.POST,
    body_parameters = {
        "from": [
            {
                "identifierTypes": [
                    "RIC"
                ],
                "values": [
                    "MSFT.O",
                    "IBM.N"
                ]
            }
        ],
        "to": [
            {
                "identifierTypes": [
                    "LEI",
                    "ISIN",
                    "ExchangeTicker"
                ]
            }
        ],
        "type": "auto"
    }
)
response = request_definition.get_data()
response.data.raw

TypeError: __init__() got an unexpected keyword argument 'body_parameters'

### Checking the Response Data
How to check if we got a sucessful Response and the data formats it contains

In [ ]:
# Check if request was successful
if (response.is_success):
    # Should have raw data when is_success==true
    if response.data.raw:
        display(response.data.raw)
    else:
        print("Response does not contain raw data")    
    # Check if the response contains a Dataframe too?
    if (not response.data.df.empty):
        display(response.data.df)
    else:
        print("\nResponse does not contain DataFrame")
else: 
    # Something went wrong
    print(f"Not Sucessful : {response.http_status}")